#### 붓꽃 품종 분류
 - 목표 : 붓꽃의 3개 품종 분류하기
 - 데이터셋 : 내장 데이터셋 사용
 - 피쳐 : 4개
 - 타겟 : 1개(품종)
 - 학습 : 지도학습 > 분류

[1] 데이터 준비

In [422]:
# 모듈로딩
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


In [423]:
# 내장 데이터셋 로딩
data = load_iris(as_frame=True)

In [424]:
# Bunch 인스턴스 => dict와 유사한 형태
data.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])

In [425]:
featureDF = data['data']
targetSR = data['target']

In [426]:
featureDF.shape, targetSR.shape

((150, 4), (150,))

[2] 데이터셋 준비
 - 학습용, 검증용, 테스트용

In [427]:
# 학습용, 테스트용 분리
X_train, X_test, y_train, Y_test = train_test_split(featureDF, targetSR, 
                                                    stratify=targetSR)

In [428]:
# 학습용, 검증용 분리(X_train 가져와서 분리)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, 
                                                  stratify=y_train)

In [429]:
print(f'Train DS : {X_train.shape[0]}   {X_train.shape[0]/featureDF.shape[0]}%')
print(f'val DS : {X_val.shape[0]}   {X_val.shape[0]/featureDF.shape[0]}%')
print(f'Test DS : {X_test.shape[0]}   {X_test.shape[0]/featureDF.shape[0]}%')

Train DS : 84   0.56%
val DS : 28   0.18666666666666668%
Test DS : 38   0.25333333333333335%


[3] 교차검증 방식

In [430]:
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier

In [431]:
# 모델 인스턴스 생성
dtc_model = DecisionTreeClassifier()

# [3-1] FKold 기반 ---------------------------
# 정확도 저장 리스트
accuracys = []

# KFold 인스턴스 생성 [기본 k=5]
kfold = KFold()

In [432]:
# k의 개수만큼 데이터셋으로 학습 반복
# k등분 후 학습용 데이터셋 인덱스, 검증용 데이터셋 인덱스 반환
for idx, (train_index, val_index) in enumerate(kfold.split(featureDF),1):

    #print(f'train_index : {train_index.tolist()}')

    # X_train, X_val 데이터셋 설정
    X_train,y_train = featureDF.iloc[train_index.tolist()], targetSR[train_index.tolist()]
    X_val,y_val = featureDF.iloc[val_index.tolist()], targetSR[val_index.tolist()]

    #학습 진행
    dtc_model.fit(X_train,y_train)

    # 평가 => 분류의 경우 score() 메서드 => 정확도 반환
    train_acc = dtc_model.score(X_train,y_train)
    val_acc = dtc_model.score(X_val,y_val)

    accuracys.append([train_acc,val_acc])
    print(f'{idx}번째 Train 정확도 : {train_acc} Val 정확도 : {val_acc}')

1번째 Train 정확도 : 1.0 Val 정확도 : 1.0
2번째 Train 정확도 : 1.0 Val 정확도 : 1.0
3번째 Train 정확도 : 1.0 Val 정확도 : 0.8666666666666667
4번째 Train 정확도 : 1.0 Val 정확도 : 0.9333333333333333
5번째 Train 정확도 : 1.0 Val 정확도 : 0.7333333333333333


In [433]:
# 평균 계산
train_mean = sum([value[0] for value in accuracys])/kfold.n_splits
test_mean = sum([value[1] for value in accuracys])/kfold.n_splits

print(f'Train 정확도 : {train_mean}   val 정확도 : {test_mean:.2f}')

Train 정확도 : 1.0   val 정확도 : 0.91


In [434]:
# [3-2] stratifiedFold : 정답/레이블/타겟의 비율을 고려해서 데이터 나눠줌

accuracys=[]

skFold = StratifiedKFold()
# k의 개수만큼 데이터셋으로 학습 반복
# k등분 후 학습용 데이터셋 인덱스, 검증용 데이터셋 인덱스 반환
for idx, (train_index, val_index) in enumerate(skFold.split(featureDF,targetSR),1):

    #print(f'train_index : {train_index.tolist()}')

    # X_train, X_val 데이터셋 설정
    X_train,y_train = featureDF.iloc[train_index.tolist()], targetSR[train_index.tolist()]
    X_val,y_val = featureDF.iloc[val_index.tolist()], targetSR[val_index.tolist()]

    #학습 진행
    dtc_model.fit(X_train,y_train)

    # 평가 => 분류의 경우 score() 메서드 => 정확도 반환
    train_acc = dtc_model.score(X_train,y_train)
    val_acc = dtc_model.score(X_val,y_val)

    accuracys.append([train_acc,val_acc])
    print(f'{idx}번째 Train 정확도 : {train_acc} Val 정확도 : {val_acc}')

1번째 Train 정확도 : 1.0 Val 정확도 : 0.9666666666666667
2번째 Train 정확도 : 1.0 Val 정확도 : 0.9666666666666667
3번째 Train 정확도 : 1.0 Val 정확도 : 0.9
4번째 Train 정확도 : 1.0 Val 정확도 : 1.0
5번째 Train 정확도 : 1.0 Val 정확도 : 1.0


In [435]:
# 평균 계산
train_mean = sum([value[0] for value in accuracys])/kfold.n_splits
test_mean = sum([value[1] for value in accuracys])/kfold.n_splits

print(f'Train 정확도 : {train_mean}   val 정확도 : {test_mean:.2f}')

Train 정확도 : 1.0   val 정확도 : 0.97


교차검증 및 성능평가 동시 진행 함수 
- cross_val_score
- cross_val_predict

In [436]:
from sklearn.model_selection import cross_val_score, cross_val_predict, cross_validate

In [437]:
### [1] 전체 DS ==> 학습용과 테스트용 DS 분리
X_train, X_test, y_train, y_test = train_test_split(featureDF,targetSR,
                                                    stratify=targetSR,
                                                    random_state=10)

In [438]:
### corss_val_predict
predict = cross_val_predict(dtc_model,X_train,y_train,cv=3)

In [439]:
print(f'predict : {predict}')

predict : [0 2 1 0 1 1 2 1 1 1 2 0 2 2 1 2 0 2 0 2 0 1 1 1 0 1 0 2 2 2 1 2 0 0 0 1 0
 0 0 1 1 2 0 2 1 0 2 1 0 0 2 0 2 0 0 0 1 1 1 0 1 2 1 0 1 1 2 1 2 0 1 2 1 0
 2 2 1 1 0 2 2 1 1 2 1 2 1 2 0 0 0 0 0 2 1 2 2 2 0 0 2 0 1 2 2 2 0 1 0 2 0
 1]


In [440]:
### cross_val_score
score = cross_val_score(dtc_model,X_train,y_train, cv=10)
score

array([1.        , 0.91666667, 1.        , 0.81818182, 0.90909091,
       0.90909091, 1.        , 0.90909091, 1.        , 0.81818182])

In [441]:
### cross_validate
'''
estimator : 각 순서의 값을 구할 때 사용된 모델
estimator의 사용법 : 각 score가 구해졌을 때 사용된 모델이 명시되어 있으니 최고의 score를 출력한 모델이 무엇인지 알 수 있음
이후 해당 모델을 사용하면 됨
모델의 형태는 변하지 않지만 그 안의 값이 변함
'''
result = cross_validate(dtc_model,X_train,y_train, 
                        return_train_score=True,
                        return_estimator=True)
resultDF= pd.DataFrame(result).loc[:,['test_score','train_score']]
resultDF

,test_score,train_score
0,0.956522,1.0
1,0.956522,1.0
2,0.954545,1.0
3,0.863636,1.0
4,0.909091,1.0


In [442]:
# best_model
best_model = result['estimator'][1]
best_model

DecisionTreeClassifier()

테스트 데이터로 확인

In [443]:
best_model.predict(X_test)

array([1, 1, 2, 2, 2, 1, 0, 2, 2, 0, 1, 1, 1, 1, 0, 2, 0, 1, 0, 0, 2, 2,
       0, 1, 2, 0, 2, 2, 1, 2, 0, 0, 0, 1, 1, 1, 0, 2])

In [445]:
best_model.score(X_test,y_test)

1.0

1. 전처리
2. 학습용 테스트용 분리
3. cross_validate(자동교차검증)로 최적의 모델 찾기
4. 테스트 데이터로 확인
